In [33]:
import time
import nltk
import pandas as pd
from stop_words import get_stop_words
import multiprocessing as mp
import re
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = get_stop_words('en') 

[nltk_data] Downloading package punkt to C:\Users\Drew
[nltk_data]     Meseck\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Drew Meseck\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [24]:
df = pd.read_csv('data/spam_review_data.csv', engine= 'python')

In [25]:
df.head()

,Unnamed: 0,text,polarity,truthful
0,0,"After recent week stay at the Affinia Hotels, ...",pos,dec
1,1,"Although much too overpriced in my opinion, th...",pos,dec
2,2,The Affinia hotel in Chicago was superb. the r...,pos,dec
3,3,THIS HOTEL IS FANTASTIC. I stayed there on my ...,pos,dec
4,4,The Affinia Chicago is a wonderful place to st...,pos,dec


In [26]:
def tokenize_data(text):
    text = text.lower()
    text = re.sub('[!@#$%^&*-+=_]', '', text)
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if len(word) > 2]
    #pbar.update(1)
    return text
    

In [27]:
def tknz(col):
    return [tokenize_data(row) for row in col]


In [28]:
df['tokens'] = tknz(df.text)

In [29]:
df.to_csv("data/train.csv")

In [30]:
df.head()

,Unnamed: 0,text,polarity,truthful,tokens
0,0,"After recent week stay at the Affinia Hotels, ...",pos,dec,"[recent, week, stay, affinia, hotels, can, def..."
1,1,"Although much too overpriced in my opinion, th...",pos,dec,"[although, much, overpriced, opinion, hotel, s..."
2,2,The Affinia hotel in Chicago was superb. the r...,pos,dec,"[affinia, hotel, chicago, superb, room, servic..."
3,3,THIS HOTEL IS FANTASTIC. I stayed there on my ...,pos,dec,"[hotel, fantastic, stayed, way, chicago, towar..."
4,4,The Affinia Chicago is a wonderful place to st...,pos,dec,"[affinia, chicago, wonderful, place, stay, hus..."


In [39]:
vec = TfidfVectorizer()
doc = [" ".join(i) for i in df.tokens]
X = vec.fit_transform(doc)

In [42]:
X[2]

<1x9423 sparse matrix of type '<class 'numpy.float64'>'
	with 48 stored elements in Compressed Sparse Row format>

In [43]:
X_train, X_test, y_train, y_test  = train_test_split(
    X, df.truthful, test_size= .33, random_state = 42)

In [44]:
clf = RandomForestClassifier(max_depth=2, random_state = 0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)